In [ ]:
import sys

sys.path.append('..')

from functions import get_confusion_matrix, get_validity_metrics
import pandas as pd

sensor = 'trunk'

df = pd.read_parquet(f'processed_{sensor}.parquet')

annotations = df['ground_truth'].unique().tolist()

# df = df[~df['ground_truth'].isin(["shuffling", "cycling (stand, inactive)", "cycling (stand)"])]

rename = {
    'standing': 'stand',
    'shuffling': 'stand',
    'walking': 'walk',
    'stairs (descending)': 'stairs',
    'stairs (ascending)': 'stairs',
    'sitting': 'sit',
    'cycling (sit)': 'bicycle',
    'lying': 'lie',
    'cycling (sit, inactive)': 'bicycle',
    'cycling (stand)': 'bicycle',
    'running': 'run',
    'cycling (stand, inactive)': 'bicycle',
}
df['ground_truth'] = df['ground_truth'].map(rename)

df.loc[df['activity'] == 'move', 'activity'] = 'stand'
# df.loc[df['activity'] == 'stairs', 'activity'] = 'walk'

plot = get_confusion_matrix(
    df['ground_truth'], df['activity'], labels=['lie', 'sit', 'stand', 'walk', 'stairs', 'run', 'bicycle']
)  # move, stairs
plot.write_image(f'{sensor}.png', scale=2)

results = get_validity_metrics(df['ground_truth'], df['activity'])
results.to_csv(f'{sensor}.csv')